In [1]:
import torch
import os, sys
from datetime import datetime

sys.path.insert(0, 'src')
from train import main as train_fn
from predict import predict
from parse_config import ConfigParser
import datasets.datasets as module_data
from utils.utils import read_json, ensure_dir, informal_log
from utils.model_utils import prepare_device
from model import metric as module_metrics
from model import loss as module_loss

### Load hparam search variables

In [2]:
config_path = 'configs/train_ade20k_explainer_KD.json'
debug = False
if debug:
    learning_rates = [1e-6] #, 1e-5, 1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1] #, 1e-2, 1e-3]
else:
    learning_rates = [1e-6, 1e-5, 1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1, 1e-2, 1e-3]

config_json = read_json(config_path)


### Create train and validation datasets outside of loop

In [3]:
dataset_args = config_json['dataset']['args']
train_dataset = module_data.KDDataset(split='train', **dataset_args)
val_dataset = module_data.KDDataset(split='val', **dataset_args)

dataloader_args = config_json['data_loader']['args']
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    **dataloader_args)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    shuffle=False,
    **dataloader_args)

In [ ]:
best = {
    'lr': -1,
    'wd': -1,
    'val_acc': -1
}
n_trials = len(learning_rates) * len(weight_decays)
trial_idx = 1
timestamp = datetime.now().strftime(r'%m%d_%H%M%S')

# Logging
log_path = os.path.join(config_json['trainer']['save_dir'], timestamp, 'log.txt')
ensure_dir(os.path.dirname(log_path))
informal_log("Hyperparameter search", log_path)
informal_log("Learning rates: {}".format(learning_rates), log_path)
informal_log("Weight decays: {}".format(weight_decays), log_path)

# Debug mode
if debug:
    config_json['trainer']['epochs'] = 1
    
for lr in learning_rates:
    for wd in weight_decays:
        # Update config json
        config_json['optimizer']['args'].update({
            'lr': lr,
            'weight_decay': wd
        })
        
        # Create run ID for trial
        itr_timestamp = datetime.now().strftime(r'%m%d_%H%M%S')
        informal_log("[{}] Trial {}/{}: LR = {} WD = {}".format(
            itr_timestamp, trial_idx, n_trials, lr, wd), log_path)
        run_id = os.path.join(timestamp, 'trials', 'lr_{}-wd_{}'.format(lr, wd))
        config = ConfigParser(config_json, run_id=run_id)
        print(config.config['optimizer']['args'])
        
        # Train model
        model = train_fn(
            config=config, 
            train_data_loader=train_dataloader,
            val_data_loader=val_dataloader)
        
        # Restore model
        model_restore_path = os.path.join(config.save_dir, 'model_best.pth')
        
        model.restore_model(model_restore_path)
        print("restored model")
        # Run on validation set using predict function
        device, device_ids = prepare_device(config_json['n_gpu'])
        metric_fns = [getattr(module_metrics, met) for met in config_json['metrics']]
        loss_fn = getattr(module_loss, config_json['loss'])
        trial_path = os.path.dirname(os.path.dirname(model_restore_path))
        output_save_path = os.path.join(trial_path, "val_outputs.pth")
        log_save_path = os.path.join(trial_path, "val_metrics.pth")
        
        validation_data = predict(
            data_loader=val_dataloader,
            model=model,
            metric_fns=metric_fns,
            device=device,
            loss_fn=loss_fn,
            output_save_path=output_save_path,
            log_save_path=log_save_path)
       
        # Obtain accuracy and compare to previous best
        print(validation_data['metrics'].keys())
        val_accuracy = validation_data['metrics']['accuracy']
        if val_accuracy > best['val_acc']:
            best.update({
                'lr': lr,
                'wd': wd,
                'val_acc': val_accuracy
            })
            informal_log("Best accuracy of {:.3f} with lr={} and wd={}".format(val_accuracy, lr, wd), log_path)
            informal_log("Trial path: {}".format(trial_path), log_path)
            # Copy model and outputs to 1 directory for easy access
            best_save_dir = os.path.join(os.path.dirname(os.path.dirname(trial_path)), 'best')
            ensure_dir(best_save_dir)
            best_outputs_save_path = os.path.join(best_save_dir, 'outputs.pth')
            best_model_save_path = os.path.join(best_save_dir, 'model.pth')
            torch.save(validation_data['logits'], best_outputs_save_path)
            model.save_model(best_model_save_path)
            informal_log("Saved model and outputs to {}".format(best_save_dir), log_path)
            
            
        trial_idx += 1

Hyperparameter search
Learning rates: [1e-06, 1e-05, 0.0001, 0.001, 0.05, 0.01, 0.5, 0.1]
Weight decays: [0, 0.1, 0.01, 0.001]
[0523_164052] Trial 1/32: LR = 1e-06 WD = 0
OrderedDict([('lr', 1e-06), ('weight_decay', 0), ('amsgrad', False)])
Created LinearLayers model with 19216 trainable parameters
Training from scratch.
Checkpoint save directory: saved/PlacesCategoryClassification/0510_102912/ADE20K_predictions/saga/KD_baseline_explainer/hparam_search/0523_164052/trials/lr_1e-06-wd_0/models
Train Epoch: 1 [0/53 (0%)] Loss: 2.764108
Train Epoch: 1 [48/53 (91%)] Loss: 2.758164
    epoch          : 1
    val_TP         : [  0   8 227   0  74   3  53   3   0   1  20   6  12   0  40   1]
    val_TN         : [3989 4049 3098 4301 3256 4089 3524 4196 4120 4331 3973 3578 4166 4178
 3926 3862]
    val_FPs        : [ 155   80  168   13 1066  127  614  168   69   65  200  545  170  234
  291   29]
    val_FNs        : [298 305 949 128  46 223 251  75 253  45 249 313  94  30 185 550]
    val_accu

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 2 [0/53 (0%)] Loss: 2.758502
Train Epoch: 2 [48/53 (91%)] Loss: 2.754575
    epoch          : 2
    val_TP         : [  0   8 228   0  74   3  53   3   0   1  22   6  12   0  42   1]
    val_TN         : [3985 4048 3095 4301 3263 4090 3525 4197 4120 4333 3972 3577 4167 4181
 3926 3861]
    val_FPs        : [ 159   81  171   13 1059  126  613  167   69   63  201  546  169  231
  291   30]
    val_FNs        : [298 305 948 128  46 223 251  75 253  45 247 313  94  30 183 550]
    val_accuracy   : 0.10198108959927961
    val_per_class_accuracy: [0.89711842 0.91310221 0.74808645 0.96825754 0.75123818 0.92143179
 0.80549302 0.94552004 0.92751013 0.97568663 0.89914453 0.80661864
 0.94079244 0.94124268 0.89329131 0.86942819]
    val_per_class_accuracy_mean: 0.8877476361999099
    val_precision  : [0.         0.08988764 0.57142857 0.         0.06531333 0.02325581
 0.07957958 0.01764706 0.         0.015625   0.09865471 0.01086957
 0.06629834 0.         0.12612613 0.03225806]
    val

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 3 [0/53 (0%)] Loss: 2.757798
Train Epoch: 3 [48/53 (91%)] Loss: 2.760552
    epoch          : 3
    val_TP         : [  0   9 236   0  73   3  53   3   0   1  23   6  12   0  43   1]
    val_TN         : [3984 4047 3089 4303 3269 4094 3527 4199 4120 4334 3971 3576 4169 4183
 3925 3861]
    val_FPs        : [ 160   82  177   11 1053  122  611  165   69   62  202  547  167  229
  292   30]
    val_FNs        : [298 304 940 128  47 223 251  75 253  45 246 313  94  30 182 550]
    val_accuracy   : 0.10423232778027916
    val_per_class_accuracy: [0.89689329 0.91310221 0.7485367  0.96870779 0.7523638  0.92233228
 0.80594327 0.94597028 0.92751013 0.97591175 0.89914453 0.80639352
 0.94124268 0.94169293 0.89329131 0.86942819]
    val_per_class_accuracy_mean: 0.8880290409725349
    val_precision  : [0.         0.0989011  0.57142857 0.         0.06483126 0.024
 0.07981928 0.01785714 0.         0.01587302 0.10222222 0.01084991
 0.06703911 0.         0.12835821 0.03225806]
    val_prec

...

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 11 [0/53 (0%)] Loss: 2.754046
Train Epoch: 11 [48/53 (91%)] Loss: 2.758987
    epoch          : 11
    val_TP         : [  0  12 314   0  72   3  54   3   0   1  26   9  12   0  47   1]
    val_TN         : [3987 4027 3057 4305 3342 4101 3541 4219 4126 4346 3954 3575 4184 4211
 3903 3864]
    val_FPs        : [157 102 209   9 980 115 597 145  63  50 219 548 152 201 314  27]
    val_FNs        : [298 301 862 128  48 223 250  75 253  45 243 310  94  30 178 550]
    val_accuracy   : 0.12471859522737505
    val_per_class_accuracy: [0.89756866 0.9092751  0.75889239 0.96915804 0.76857271 0.92390815
 0.80932013 0.95047276 0.92886087 0.97861324 0.8959928  0.80684376
 0.94461954 0.9479964  0.88923908 0.87010356]
    val_per_class_accuracy_mean: 0.8905898244034218
    val_precision  : [0.         0.10526316 0.60038241 0.         0.06844106 0.02542373
 0.08294931 0.02027027 0.         0.01960784 0.10612245 0.01615799
 0.07317073 0.         0.13019391 0.03571429]
    val_precision_mea

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 12 [0/53 (0%)] Loss: 2.753898
Train Epoch: 12 [48/53 (91%)] Loss: 2.755840
    epoch          : 12
    val_TP         : [  0  13 321   0  72   3  54   3   0   0  26   9  12   0  48   2]
    val_TN         : [3985 4024 3054 4305 3350 4101 3545 4220 4127 4347 3952 3574 4186 4215
 3903 3863]
    val_FPs        : [159 105 212   9 972 115 593 144  62  49 221 549 150 197 314  28]
    val_FNs        : [298 300 855 128  48 223 250  75 253  46 243 310  94  30 177 549]
    val_accuracy   : 0.12674470959027465
    val_per_class_accuracy: [0.89711842 0.90882485 0.75979289 0.96915804 0.77037371 0.92390815
 0.81022062 0.95069788 0.929086   0.97861324 0.89554255 0.80661864
 0.94506979 0.94889689 0.88946421 0.87010356]
    val_per_class_accuracy_mean: 0.8908430886987844
    val_precision  : [0.         0.11016949 0.60225141 0.         0.06896552 0.02542373
 0.08346213 0.02040816 0.         0.         0.10526316 0.01612903
 0.07407407 0.         0.13259669 0.06666667]
    val_precision_mea

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))
